# **Computational Creativity Project**

---

### **Recipe generation and image generation based on the generated recipe using LSTM and Big Sleep**

Samil Enes Basbug, Student ID: 210617146


## Important information about this notebook:

In this project, I built a text generation to generate chicken, meat, and fish recipes. Then, I utilized the Big Sleep model to generate images related to the recipes my model created. Please do not skip any line of code(run each piece of code respectively), I have already commented the training part out. The notebook is ready to be tested.

1.   I built and deployed an web app, using Streamlit (is an open-source app framework), generating food recipes. I recommend you to visit this web app as well :)  https://share.streamlit.io/enesbasbug/food_recipe_generator/main/app.py

 (The only model embedded in the web app was Recipe(text) generation model. I could not work with Big Sleep locally since CUDA only works with Nvidia graphic card I did not have. Therefore, even though I integrated Big Sleep to my Text Generation model on my computer, Big Sleep did not work.) 
2.   You may also visit my GitHub https://github.com/enesbasbug/Food_Recipe_Generator repository to see many texts/images/gifs examples of my project.



What you will see in this notebook respectively:
*   Installing the OpenAI's CLIP and a BigGAN model, also known as Big Sleep.
*   Importing the libraries necessary.
*   Loading the pre-trained model for text/recipe generator.
*   Loading the data, data cleaning and creating the files contain new data, which will be used by the text generator model
*   Reading the data
*   The common functions and variables that we will use
*   Generating the Text/Recipe (You will see there are 3 similar, in order, blocks of codes to generate the recipes since there are 3 specific pre-trained models for fish, meat and chicken dishes.) You may execute and try them by yourself
*   Determining the ingredients in the recipes
*   Image generation from the recipe/text using Big Sleep. You may execute and try it by yourself.





# Installing the Big Sleep Model for image generation and importing the libraries necessary
- We will generate images using the generated text/recipes as inputs in last step

In [ ]:
!pip install big-sleep

     |████████████████████████████████| 1.4 MB 6.6 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 87 kB 7.8 MB/s 
     |████████████████████████████████| 132 kB 89.2 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 8.7 MB 50.9 MB/s 
     |████████████████████████████████| 138 kB 59.7 MB/s 
     |████████████████████████████████| 127 kB 61.9 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=97660fda6a858c2e3a50341f73b338408435970a072f29d9e065e500668180ce
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout, Bidirectional
from tensorflow.keras import regularizers
from tensorflow.keras.losses import sparse_categorical_crossentropy

from tqdm.notebook import trange
from IPython.display import Image, display

from big_sleep import Imagine

# Load pre-trained data from Google Drive 
- Pre-trained (70 Epochs) Models for Chicken, Meat and Fish recipes.
- Please, when you run this line of code check the left side where these downloaded models need to be. Sometimes the last model would not be there, which means it has not been downloaded. In this case, rerun this line of code, please. There supposed to be 3 different models named chicken_model.h5, meat_model.h5 and fish_model.h5. Then, you can see all of them there.



In [ ]:
! gdown -- 1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv # https://drive.google.com/file/d/1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv/view?usp=sharing # model for chicken dish 

! gdown -- 1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo # https://drive.google.com/file/d/1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo/view?usp=sharing # model for meat dish

! gdown -- 1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz # https://drive.google.com/file/d/1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz/view?usp=sharing # model for fish dish

Downloading...
From: https://drive.google.com/uc?id=1TtC9S4b5KSGcDeVaLzaciAr7rCwO4Bgv
To: /content/chicken_model.h5
100% 164M/164M [00:03<00:00, 48.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s9Tw0gOfaNnfZTS3stDp6lpk4UeqfuAo
To: /content/meat_model.h5
100% 164M/164M [00:03<00:00, 43.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h6xCW_gpfunhAmXZ9TonC6Xr-Q_Nz7Tz
To: /content/fish_model.h5
100% 164M/164M [00:03<00:00, 42.2MB/s]


#Data Cleaning

In [ ]:
import pandas as pd
#@title Reading data stored in CSV files  
# I did not get the data from Kaggle directly since it required API token, which would be a problem for the reader/user. 
# Therefore, I upload the raw data to my GitHub repository and call it from here.

url = 'https://raw.githubusercontent.com/enesbasbug/Generate_image_from_generated_text/main/data_recipe.csv'
# just in case, you can comment out above and use the url below 
# url = 'https://raw.githubusercontent.com/enesbasbug/Food_Recipe_Generator/main/data_recipe.csv' ## https://github.com/enesbasbug/Food_Recipe_Generator/blob/main/data_recipe.csv

df = pd.read_csv(url, index_col=0)
df.head(2)

,category,cooking_method,cuisine,image,ingredients,prep_time,recipe_name,serves,tags
0,NaN,['Set the racks in the middle and upper thirds...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['1 tablespoons extra virgin olive oil', '1 cu...",20 minutes,Mediterranean Sea Bass,4 servings,"Dairy Free, Gluten Free, Keto Recipes, Kid Fri..."
1,NaN,['Place the eggs in the air fryer basket and c...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['4 large eggs', 'Salt (black pepper, everythi...",15 minutes,Air Fryer Hard Boiled Eggs,4 eggs,"Air Fryer Recipes, Dairy Free, Gluten Free, Ke..."


In [ ]:
#@title Removing unnecessary features of the data
df.drop(columns=['category', 'cuisine', 'image', 'prep_time', 'serves', 'tags', 'recipe_name'], inplace=True)

In [ ]:
df.head(2)

,cooking_method,ingredients
0,['Set the racks in the middle and upper thirds...,"['1 tablespoons extra virgin olive oil', '1 cu..."
1,['Place the eggs in the air fryer basket and c...,"['4 large eggs', 'Salt (black pepper, everythi..."


In [ ]:
#@title Cleaning the data
for i in df.columns:
    df[i].replace('[]', np.nan, inplace=True)

for i in df.columns:
    df.dropna(subset=[i], inplace=True)

df = df.reset_index(drop=True)

In [ ]:
#@title Categorizing the recipes in the date base on its type of meat such as chicken, meat and fish
df['chicken'] = [1 if 'chicken' in s.lower() else 0 for s in df['ingredients']]

df['meat'] = [1 if ('beef' or 'meat' or 'pork' or 'bacon') in s.lower() else 0 for s in df['ingredients']]
                            
df['fish'] = [1 if ('fish' or 'salmon' or 'sea bass') in s.lower() else 0 for s in df['ingredients']]

In [ ]:
print(len(df.chicken))
print(len(df.meat))
print(len(df.fish))

56312
56312
56312


In [ ]:
#@title Text adjustments

import re
from string import punctuation
# I wanted to keep ',' and '.' in the text so that the generated text/recipe seems real, accurate and easy to read.
# The only punctuations the text have dot and comma.
my_punctuation = punctuation.replace(".","")
my_punctuation = my_punctuation.replace(",","")


c = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['chicken'][i] == 1)]
c = ''.join(c)
chicken = c.translate(str.maketrans("", "", my_punctuation))

m = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['meat'][i] == 1)]
m = ''.join(m)
meat = m.translate(str.maketrans("", "", my_punctuation))

f = [df['cooking_method'][i].lower() for i in range(len(df)) if (df['fish'][i] == 1)]
f = ''.join(f)
fish = f.translate(str.maketrans("", "", my_punctuation))

ingredients = [df['ingredients'][i].lower() for i in range(len(df))]
ingredients = ''.join(f)
ingredients = f.translate(str.maketrans("", "", punctuation))



In [ ]:
#@title I limited the number of characters in the text files
fish = fish[:500000]
chicken = chicken[:500000]
meat = meat[:500000]
ingredients = ingredients[:10000]

In [ ]:
print(len(fish))
print(len(meat))
print(len(fish))
print(len(ingredients))

500000
500000
500000
10000


In [ ]:
#@title Exporting the data for fish, meat and chicken
file_f = open("fish.txt","w")#write mode
file_f.write(fish)
file_f.close()

file_m = open("meat.txt","w")#write mode
file_m.write(meat)
file_m.close()

file_c = open("chicken.txt","w")#write mode
file_c.write(chicken)
file_c.close()

file_i = open("ingredients.txt","w")#write mode
file_i.write(ingredients)
file_i.close()

# Reading the data

In [ ]:
TEXT_chicken = open('chicken.txt', 'rb').read().decode(encoding='utf-8')
TEXT_meat = open('meat.txt', 'rb').read().decode(encoding='utf-8')
TEXT_fish = open('fish.txt', 'rb').read().decode(encoding='utf-8')
TEXT_ingredients = open('ingredients.txt', 'rb').read().decode(encoding='utf-8')
# checking the number of characters in it
print(len(TEXT_chicken))
print(len(TEXT_meat))
print(len(TEXT_fish))
print(len(TEXT_ingredients))

500000
500000
500000
10000


# Common functions that recipe generation model uses

In [ ]:
#@title Common functions we will use for recipe generation
# Reference: Some of functions are similar to the week-3 Lab notebook provided for the Computational Creativity Lab.

def split_input_target(chunk):
    input_TEXT = chunk[:-1]
    target_TEXT = chunk[1:]
    return input_TEXT, target_TEXT

def loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
    model.add(LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    model.add(LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dropout(0.2))
    
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=loss) 
    # print(model.summary())
    return model

def generate_text(model, start_string,t):
    
    # Number of characters to generate
    num_generate = 1000
    # vectorizing - Converting our start string to numbers 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    # Empty string to store our results
    text_generated = []
    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))


# Tring to find items in the recipes; however, it was not that successfull due to the
# the ingredients in the data was long terms instead of one words. Therefore, I could not
# compare the items in the generated recipe with the items in the ingrediens text file.
# for instance, in the ingredients data there was '1 spoon of olive oil' instead of just 'olive oil'
# so that i could not extract the items in generated recipe properly.
def word_extraction(text):
  ignore = ["a", "the", "is", "to", "until","about","with","and","as","add","on","in","at","of","over","around","knife","sharp","pairing","once","comes","for","minutes","hours"]
  words = re.sub("[^\w]", " ",  text).split()
  cleaned_text = [w.lower() for w in words if w not in ignore]
  output_ingredients = ["".join(a) for a in cleaned_text if a in ingredients]
  return output_ingredients


# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)



# Parameters for the training model

In [ ]:
# pre-trained models utilized these parameters below

BATCH_SIZE = 64  # @param {type: "integer"}

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000 # @param {type: "integer"}
# The embedding dimension
embedding_dim = 256 # @param {type: "integer"}
# Number of RNN units
rnn_units = 1024 # @param {type: "integer"}
EPOCHS = 70 # @param {type: "integer"} 

# Generating Fish Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_fish))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_fish])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)



# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("fish_model.h5")

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("fish_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
fish_recipe = generate_text(model, start_string=u"fish ",t=0.1)
# fish_recipe

In [ ]:
fish_recipe

'fish and bake it in a preheated 375 degree f oven, covered with foil, until the fish just flakes about 15 minutes., with a slotted spoon, transfer meat to a platter. reduce heat to medium low and cook shallots until well browned, about 15 minutes. add garlic and ginger stir fry 30 seconds on a baking sheet and bake until warm, 3 to 4 minutes., top the bottom half of each baguette with a pork cutlet halve the cutlets for fit, if necessary and some of the cucumber, onion, carawaypickled cauliflower, watercress and cilantro. top each with a drizzle of garlic fish sauce and a squirt of sriracha and serve.put the potatoes in a pot and cover with the clam juice. bring to a boil over high heat then reduce to mediumlow and simmer for 15 minutes, skimming and discarding any fillets in the oil and fry until golden, 4 to 6 minutes., place each fillet in the reserved melted butter so that both sides are well coated. then, mix thoroughly all the spices for blackening mixture and sprinkle the black

In [ ]:
file_ff = open("generated_fish_recipe.txt","w")#write mode
file_ff.write(fish_recipe)
file_ff.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe may look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

new_fish_recipe = generate_text(model, start_string=u"fish ",t=0.1)
new_fish_recipe

'fish and remove half of the satay for serving. add about 1 cup of the broth to the remaining satay in the skillet and bring to a simmer. cook 5 to 8 minutes, or until the fish is cooked through.in a bowl, mix the marinade ingredients except the chicken. slice the chicken into long, thin strips, about 3 inches long by 1 inch wide, and add to marinade. turn chicken to coat, cover and refrigerate at least 4 hours or longer., meanwhile, make the dressing whisk the lime juice, oil, salt and pepper in a food processor fitted with a steel blade and pulse to finely chop. add the vegetables to the stock pot and saute them along with the fish bones for 15 minutes to sweat and caramelize the vegetables. add the tomato, saffron and other herbs and seasonings and continue to saute for 10 minutes. stir to ensure that nothing burns or sticks to the bottom of the pan. add the chicken stock, fish sauce, star anise and cinnamon. bring to boil over medium heat and cook 5 minutes., 3. turn off the heat a

# Generating Meat Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_meat))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_meat])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
    

# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("meat_model.h5")
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("meat_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
meat_recipe = generate_text(model, start_string=u"meat ",t=0.1)
# meat_recipe

In [ ]:
meat_recipe

'meat is fork tender. check the seasoning. serve on buttered bowtie noodles.in a 6quart slow cooker combine the cheddar and bake until melted, 2 to 3 more minutes. serve with the vegetables.1. preheat oven to 375 degrees f., in a large skillet over mediumhigh heat, turning once, 4 minutes or until desired doneness. remove steaks from marinade and discard marinade. with a sharp knife cut a counter pan and fold it into a semicircle in the short ribs on a cutting board. sprinkle with salt and pepper. spread 12cup of black rice on the beef. place 1 cup of the beef mixture over the short ends of the triangles closest to the ramekin. cover the bread, place the meat on the pan and sear the edges, or until the pastry is golden brown and crisp. transfer the mixture to a bowl and stir in the remaining 2 tablespoons cilantro set aside., remove the cooked fries from the oven but leave on the baking sheet. alternatively you may pan, if needed, and add the meatballs in a single layer, spreading them

In [ ]:
file_mm = open("generated_meat_recipe.txt","w")#write mode
file_mm.write(meat_recipe)
file_mm.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

new_meat_recipe = generate_text(model, start_string=u"meat ",t=0.1)
new_meat_recipe

'meat is fork tender. check the seasoning. serve on buttered bowtie noodles.in a 6quart slow cooker combine the cheddar and bake until melted, 2 to 3 more minutes. serve with the vegetables.1. preheat oven to 375 degrees f., in a large skillet over mediumhigh heat, turning once, 4 minutes or until desired doneness. remove steaks from marinade and discard marinade. with a sharp knife cut a couple of times do not mix completely., drain the pasta and rinse under cool water until they no longer feel sticky. cut them into quarters with the ground turkey, bread crumbs, egg, 2 tbsp of the parsley, milk, 34 tsp salt and black pepper to taste and cook for another 3 minutes., season with salt to taste and sprinkle flour over the meat, cook 1 minute turning often. 4.sprinkle the tops with the cilantro and parsley. serve with cornbread., remove the stems from the dried chilies and shake out and discard seeds. in small saucepan place chiles with enough water to cover the mixture, about 4 cups. brin

#Generatin Chicken Food Recipe

In [ ]:
# The unique characters in the file
vocab = sorted(set(TEXT_chicken))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
TEXT_as_int = np.array([char2idx[c] for c in TEXT_chicken])
seq_length = 100

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# model.save("chicken_model.h5")
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights("chicken_model.h5")
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
chicken_recipe = generate_text(model, start_string=u"chicken ",t=0.1)

# chicken_recipe

In [ ]:
chicken_recipe

'chicken on the grill., cook the chicken until well browned on both sides and firm, but not hard to the sauce with ones hands. pull off a piece the size of a meatball, use your thumb to make an immersion blender., add the cheese to the soup and whisk over medium heat until the onion is tender, about 34 minutes, then add mushrooms and cook another 23 minutes, until soup thickens, adjust salt and pepper to taste and serve.in a large deep nonstick skillet, brown the sausage over high heat until cooked and soft., drain and mash with chicken broth, sour cream, salt and pepper., preheat oven to 400f., in a large saute pan brown meat over mediumhigh heat, add the bacon fat and bacon. simmer over low heat to thicken 20 to 30 minutes. cool and store for makeahead meal., preheat the oven to 425 degrees f. remove the paper towels on the pan and replace with parchment for easy cleanup. lightly spray with olive oil spray and bake for 30 minutes turning halfway.season the pork with salt, garlic powd

In [ ]:
file_cc = open("generated_chicken_recipe.txt","w")#write mode
file_cc.write(chicken_recipe)
file_cc.close()

In [ ]:
# To generate something new please remove the #s below
# the beginning of the recipe look like each other sometimes
# however the rest of it would definitely change.
# ___________________________

new_chicken_recipe = generate_text(model, start_string=u"chicken ",t=0.1)
new_chicken_recipe

'chicken on top, about 10 minutes., cook the bacon in a large pot over low heat until all of the chicken is well coated., cook the chicken in a large pot, melt the butter over mediumlow heat. add the celery, onion, carrot and garlic, cover and cook 20 minutes, then add the sweet potato and kale and cook until the sweet potatoes are tender., drain and return to pot., put heat on a medium heat and add 1 tablespoon of the oil. fold each piece of chicken into each pepper half hours. shape the bread cubes into the slow cooker, stirring occasionally, until the liquid is absorbed, stirring often, about 2 minutes. remove from water and set aside., to the skillet, add 1 tbsp oil and cook onions and peppers, season with salt and pepper. cook on a mediumlow heat for about 23 minutes, until onions are translucent., add chopped zucchini and season with a pinch of salt and stir well, cooking an additional 23 minutes on mediumlow heat. slowly whisk in the chicken broth until well combined. or you can

# All generated recipes 

I know last 3 pieces of code where we generated recipes were nearly similar;however, I could not run them all in a loop since it threw an error saying 'ValueError: Cannot assign value to variable ' embedding/embeddings:0': Shape mismatch.The variable shape (8, 256), and the assigned value shape (40, 256) are incompatible.' I could not fix it, unfortunately.

In [ ]:
# You may see all generated recipes here in one place
recipe_param = new_chicken_recipe #@param ['new_fish_recipe', 'new_meat_recipe', 'new_chicken_recipe'] {type:"raw"}
print("Recipe: \n ", recipe_param)



Recipe: 
  chicken on top, about 10 minutes., cook the bacon in a large pot over low heat until all of the chicken is well coated., cook the chicken in a large pot, melt the butter over mediumlow heat. add the celery, onion, carrot and garlic, cover and cook 20 minutes, then add the sweet potato and kale and cook until the sweet potatoes are tender., drain and return to pot., put heat on a medium heat and add 1 tablespoon of the oil. fold each piece of chicken into each pepper half hours. shape the bread cubes into the slow cooker, stirring occasionally, until the liquid is absorbed, stirring often, about 2 minutes. remove from water and set aside., to the skillet, add 1 tbsp oil and cook onions and peppers, season with salt and pepper. cook on a mediumlow heat for about 23 minutes, until onions are translucent., add chopped zucchini and season with a pinch of salt and stir well, cooking an additional 23 minutes on mediumlow heat. slowly whisk in the chicken broth until well combined. 

# Code for finding Inredients in the generated recipe

In [ ]:
#@title # This part was for, especially, the web app https://share.streamlit.io/enesbasbug/food_recipe_generator/main/app.py
new_meat_recipe



'meat is fork tender. check the seasoning. serve on buttered bowtie noodles.in a 6quart slow cooker combine the cheddar and bake until melted, 2 to 3 more minutes. serve with the vegetables.1. preheat oven to 375 degrees f., in a large skillet over mediumhigh heat, turning once, 4 minutes or until desired doneness. remove steaks from marinade and discard marinade. with a sharp knife cut a couple of times do not mix completely., drain the pasta and rinse under cool water until they no longer feel sticky. cut them into quarters with the ground turkey, bread crumbs, egg, 2 tbsp of the parsley, milk, 34 tsp salt and black pepper to taste and cook for another 3 minutes., season with salt to taste and sprinkle flour over the meat, cook 1 minute turning often. 4.sprinkle the tops with the cilantro and parsley. serve with cornbread., remove the stems from the dried chilies and shake out and discard seeds. in small saucepan place chiles with enough water to cover the mixture, about 4 cups. brin

In [ ]:
# This part was for, especially, the web app # https://share.streamlit.io/enesbasbug/food_recipe_generator/main/app.py

# Tring to find items in the recipes; however, it was not that successfull due to the
# the ingredients in the raw data was long terms instead of one words. Therefore, I could not
# compare the items in the generated recipe with the items in the ingrediens text file(raw data)
# for instance, in the ingredients data there was '1 spoon of olive oil' instead of just 'olive oil'
# so that i could not extract the items in generated recipe properly. This can be developped for future work.

# word_extraction function is defined in the Common functions above
ingredients_meat = word_extraction(new_meat_recipe) 
ingredients_chicken = word_extraction(new_chicken_recipe)
ingredients_fish = word_extraction(new_fish_recipe)
# ## ingredients of new meat recipe
option = ingredients_meat #@param ["ingredients_meat", "ingredients_chicken", "ingredients_fish"] {type:"raw"}
print("You selected ", option) 



You selected  ['meat', 'tender', 'seasoning', 'serve', 'noodles', 'combine', 'bake', 'melted', '2', '3', 'more', 'serve', 'vegetables', '1', 'preheat', 'oven', 'f', 'large', 'skillet', 'mediumhigh', 'heat', 'turning', '4', 'or', 'desired', 'remove', 'from', 'marinade', 'marinade', 'cut', 'do', 'not', 'mix', 'rinse', 'under', 'cool', 'water', 'no', 'longer', 'cut', 'them', 'into', 'ground', 'egg', '2', 'tbsp', 'milk', 'salt', 'black', 'pepper', 'taste', 'cook', 'another', '3', 'season', 'salt', 'taste', 'sprinkle', 'flour', 'meat', 'cook', '1', 'minute', 'turning', 'often', '4', 'sprinkle', 'tops', 'cilantro', 'serve', 'remove', 'from', 'out', 'small', 'saucepan', 'place', 'water', 'cover', 'mixture', '4', 'cups', 'bring']


# Text to Image Generator using Big Sleep
- **generated images will appear in the folder where it will be saved on the left side, in order to see them please check there**

- I set iteration 100 to generate a quick image. But, I recommend you to increase the number of iteration at the bottom of the loop, which is in a loop. It increase the quality of images; however, it take longer to generate.

In [ ]:
text_to_img_fish = new_fish_recipe[:210]
text_to_img_meat = new_meat_recipe[:210]
text_to_img_chicken = new_chicken_recipe[:210]
mix = [text_to_img_fish, text_to_img_meat, text_to_img_chicken]
for T in range(3):
  TEXT = mix[T] # You may change the TEXT by yourself
               #  to try something else than recipes, 
               # but, please, be aware you are in a loop
  SAVE_EVERY = 100 #@param {type:"number"}
  SAVE_PROGRESS = True #@param {type:"boolean"}
  LEARNING_RATE = 5e-2 #@param {type:"number"}
  ITERATIONS = 1050 #@param {type:"number"}
  SEED = 0 #@param {type: "number"}
  model = Imagine(
      text = TEXT,
      save_every = SAVE_EVERY,
      lr = LEARNING_RATE,
      iterations = ITERATIONS,
      save_progress = SAVE_PROGRESS,
      seed = SEED
  )
  for epoch in trange(1, desc = 'epochs'): # number of epochs
      for i in trange(100, desc = 'iteration'): # you can decrease 
          model.train_step(epoch, i)           # 'the iteration' to achieve faster results
                                              # however, the quality of imgs may decrease too

          if i == 0 or i % model.save_every != 0:
              continue
          filename = TEXT.replace(' ', '_')
          image = Image(f'./{filename}.png')
          display(image)

setting seed of 0
you can override this with --seed argument in the command line, or --random for a randomly chosen one


100%|████████████████████████████████████████| 354M/354M [00:01<00:00, 233MiB/s]
100%|██████████| 800/800 [00:00<00:00, 508785.93B/s]


epochs:   0%|          | 0/1 [00:00<?, ?it/s]

iteration:   0%|          | 0/100 [00:00<?, ?it/s]

image updated at "./fish_and_remove_half_of_the_satay_for_serving._add_about_1_cup_of_the_broth_to_the_remaining_satay_in_the_skillet_and_bring_to_a_simmer._cook_5_to_8_minutes_or_until_the_fish_is_cooked_through.in_a_bowl_mix_t.png"
setting seed of 0
you can override this with --seed argument in the command line, or --random for a randomly chosen one


...

iteration:   0%|          | 0/100 [00:00<?, ?it/s]

image updated at "./meat_is_fork_tender._check_the_seasoning._serve_on_buttered_bowtie_noodles.in_a_6quart_slow_cooker_combine_the_cheddar_and_bake_until_melted_2_to_3_more_minutes._serve_with_the_vegetables.1._preheat_oven_to_37.png"
setting seed of 0
you can override this with --seed argument in the command line, or --random for a randomly chosen one


epochs:   0%|          | 0/1 [00:00<?, ?it/s]

iteration:   0%|          | 0/100 [00:00<?, ?it/s]

image updated at "./chicken_on_top_about_10_minutes._cook_the_bacon_in_a_large_pot_over_low_heat_until_all_of_the_chicken_is_well_coated._cook_the_chicken_in_a_large_pot_melt_the_butter_over_mediumlow_heat._add_the_celery_oni.png"


In [ ]:
# Reminder
# You may also visit my GitHub https://github.com/enesbasbug/Food_Recipe_Generator 
# repository to see many texts/images/gifs examples of my project.

In [ ]:
print('Thank you!')

Thank you!
